# Day 5: CUDA Graphs - Recipe Cards for the GPU

---

## 🎯 The Hook: The Head Chef's Secret Weapon

**Ever watched** a professional kitchen during rush hour?

They don't figure out how to make each dish from scratch every time. They have **recipe cards** - pre-planned sequences that get executed perfectly every time with minimal thinking.

```
Without Recipe Cards (Regular Launches):
┌───────────────────────────────────────────────────────────────┐
│  Order 1: "How do I make this again?" → Read recipe → Cook    │
│  Order 2: "Wait, what temperature?" → Read recipe → Cook      │
│  Order 3: "Which spices?" → Read recipe → Cook                │
│                                                               │
│  Overhead: Thinking time before EVERY dish! 🐌                │
└───────────────────────────────────────────────────────────────┘

With Recipe Cards (CUDA Graphs):
┌───────────────────────────────────────────────────────────────┐
│  Step 1: Create recipe card ONCE (capture graph)              │
│  ┌─────────────────────────────────────────────────────────┐  │
│  │  1. Prep → 2. Season → 3. Sear → 4. Rest → 5. Plate    │  │
│  └─────────────────────────────────────────────────────────┘  │
│                                                               │
│  Step 2: Execute recipe instantly, every time!                │
│  Order 1: [Execute recipe] ← No thinking, just do!           │
│  Order 2: [Execute recipe]                                    │
│  Order 3: [Execute recipe]                                    │
│                                                               │
│  Overhead: Near-ZERO per dish! 🚀                             │
└───────────────────────────────────────────────────────────────┘
```

**CUDA Graphs capture your kernel workflow** and replay it with minimal CPU overhead!

---

## 📚 Learning Objectives

By the end of this lesson, you will be able to:
1. **Explain** why CUDA Graphs reduce launch overhead dramatically
2. **Capture** existing stream operations into a graph
3. **Build graphs manually** for complex dependency patterns
4. **Update graph parameters** without rebuilding the graph
5. **Identify** when graphs provide the most benefit

---

## 🃏 Concept Card: Recipe Cards

```
╔══════════════════════════════════════════════════════════════════╗
║  📋 CUDA GRAPHS = PRE-COMPILED RECIPE CARDS                      ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  THE PROBLEM: Launch Overhead                                    ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  Each kernel launch: ~5-10μs CPU overhead                  │  ║
║  │                                                            │  ║
║  │  100 small kernels = 500-1000μs of JUST launching!         │  ║
║  │  (Often more than the actual compute time!)                │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  THE SOLUTION: CUDA Graphs                                       ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  1. CAPTURE: Record sequence of operations                 │  ║
║  │     cudaStreamBeginCapture(stream)                         │  ║
║  │     kernel1<<<...>>>(); kernel2<<<...>>>(); ...            │  ║
║  │     cudaStreamEndCapture(stream, &graph)                   │  ║
║  │                                                            │  ║
║  │  2. INSTANTIATE: Compile graph for execution               │  ║
║  │     cudaGraphInstantiate(&graphExec, graph, ...)           │  ║
║  │                                                            │  ║
║  │  3. LAUNCH: Execute entire graph at once!                  │  ║
║  │     cudaGraphLaunch(graphExec, stream)  // ~10μs total!    │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  💡 KEY INSIGHT: Capture once, replay many times!                ║
║     Perfect for ML training loops, physics simulations,          ║
║     any repetitive multi-kernel workflow!                        ║
║                                                                  ║
║  PERFORMANCE IMPACT:                                             ║
║  │ Kernels │ Regular Launch │ Graph Launch │ Speedup │          ║
║  │---------|----------------|--------------|---------|          ║
║  │    10   │     50-100μs   │    ~10μs     │   5-10x │          ║
║  │   100   │    500-1000μs  │    ~10μs     │ 50-100x │          ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## Why CUDA Graphs?

**Problem**: Each kernel launch has CPU overhead (~5-10μs per launch)

**Solution**: CUDA Graphs capture entire workflows and replay them with minimal overhead

| Approach | Launch Overhead per Kernel |
|----------|---------------------------|
| Regular launches | ~5-10 μs |
| CUDA Graph launch | ~10 μs total (for entire graph) |

In [ ]:
%%writefile cuda_graphs_basics.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel1(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] + 1.0f;
}

__global__ void kernel2(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] * 2.0f;
}

__global__ void kernel3(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] - 1.0f;
}

int main() {
    const int N = 1 << 20;
    const int blocks = (N + 255) / 256;
    
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMemset(d_data, 0, N * sizeof(float));
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // === METHOD 1: Stream Capture ===
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    // Begin capture
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    // These are captured, NOT executed yet
    kernel1<<<blocks, 256, 0, stream>>>(d_data, N);
    kernel2<<<blocks, 256, 0, stream>>>(d_data, N);
    kernel3<<<blocks, 256, 0, stream>>>(d_data, N);
    
    // End capture
    cudaStreamEndCapture(stream, &graph);
    
    // Create executable graph
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // Launch graph multiple times
    printf("Launching graph 10 times...\n");
    for (int i = 0; i < 10; i++) {
        cudaGraphLaunch(graphExec, stream);
    }
    cudaStreamSynchronize(stream);
    
    // Verify result: ((0+1)*2-1) = 1, applied 10 times
    float h_result;
    cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost);
    printf("Result after 10 graph launches: %.0f\n", h_result);
    
    // Cleanup
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc cuda_graphs_basics.cu -o cuda_graphs_basics && ./cuda_graphs_basics

## Manual Graph Construction

Build graphs programmatically with full control over node dependencies.

In [ ]:
%%writefile cuda_graphs_manual.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void initKernel(float* data, int n, float val) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = val;
}

__global__ void squareKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] * data[idx];
}

int main() {
    const int N = 1024;
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Create empty graph
    cudaGraph_t graph;
    cudaGraphCreate(&graph, 0);
    
    // Add kernel nodes manually
    cudaGraphNode_t initNode, squareNode;
    
    // Kernel parameters for initKernel
    cudaKernelNodeParams initParams = {};
    void* initArgs[] = {&d_data, (void*)&N, nullptr};
    float initVal = 3.0f;
    initArgs[2] = &initVal;
    initParams.func = (void*)initKernel;
    initParams.gridDim = dim3((N + 255) / 256);
    initParams.blockDim = dim3(256);
    initParams.sharedMemBytes = 0;
    initParams.kernelParams = initArgs;
    initParams.extra = nullptr;
    
    // Add init node (no dependencies)
    cudaGraphAddKernelNode(&initNode, graph, nullptr, 0, &initParams);
    
    // Kernel parameters for squareKernel
    cudaKernelNodeParams squareParams = {};
    void* squareArgs[] = {&d_data, (void*)&N};
    squareParams.func = (void*)squareKernel;
    squareParams.gridDim = dim3((N + 255) / 256);
    squareParams.blockDim = dim3(256);
    squareParams.sharedMemBytes = 0;
    squareParams.kernelParams = squareArgs;
    squareParams.extra = nullptr;
    
    // Add square node (depends on init)
    cudaGraphNode_t deps[] = {initNode};
    cudaGraphAddKernelNode(&squareNode, graph, deps, 1, &squareParams);
    
    // Instantiate and launch
    cudaGraphExec_t graphExec;
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    cudaGraphLaunch(graphExec, stream);
    cudaStreamSynchronize(stream);
    
    float h_result;
    cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost);
    printf("Result: init(3.0) then square = %.0f (expected 9)\n", h_result);
    
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc cuda_graphs_manual.cu -o cuda_graphs_manual && ./cuda_graphs_manual

## Graph Update (Parameter Changes)

Update graph parameters without rebuilding the entire graph.

In [ ]:
%%writefile cuda_graphs_update.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void scaleKernel(float* data, int n, float scale) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] * scale;
}

int main() {
    const int N = 1024;
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    float h_data[N];
    for (int i = 0; i < N; i++) h_data[i] = 1.0f;
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // Capture initial graph with scale = 2.0
    float scale = 2.0f;
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    scaleKernel<<<(N+255)/256, 256, 0, stream>>>(d_data, N, scale);
    cudaStreamEndCapture(stream, &graph);
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // Launch with scale = 2.0
    cudaGraphLaunch(graphExec, stream);
    cudaStreamSynchronize(stream);
    
    float h_result;
    cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost);
    printf("After scale=2.0: %.1f\n", h_result);
    
    // Update graph with new scale = 3.0
    cudaGraph_t newGraph;
    scale = 3.0f;
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    scaleKernel<<<(N+255)/256, 256, 0, stream>>>(d_data, N, scale);
    cudaStreamEndCapture(stream, &newGraph);
    
    // Update existing executable with new graph
    cudaGraphExecUpdateResultInfo updateResult;
    cudaGraphExecUpdate(graphExec, newGraph, &updateResult);
    
    if (updateResult.result == cudaGraphExecUpdateSuccess) {
        printf("Graph updated successfully!\n");
    }
    
    // Launch updated graph
    cudaGraphLaunch(graphExec, stream);
    cudaStreamSynchronize(stream);
    
    cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost);
    printf("After scale=3.0: %.1f\n", h_result);
    
    cudaGraphDestroy(newGraph);
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc cuda_graphs_update.cu -o cuda_graphs_update && ./cuda_graphs_update

## Performance Comparison

In [ ]:
%%writefile cuda_graphs_benchmark.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void tinyKernel(float* data) {
    data[threadIdx.x] += 1.0f;
}

int main() {
    float* d_data;
    cudaMalloc(&d_data, 256 * sizeof(float));
    cudaMemset(d_data, 0, 256 * sizeof(float));
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    const int KERNELS = 100;
    const int ITERATIONS = 1000;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark regular launches
    cudaEventRecord(start);
    for (int iter = 0; iter < ITERATIONS; iter++) {
        for (int k = 0; k < KERNELS; k++) {
            tinyKernel<<<1, 256>>>(d_data);
        }
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float regularMs;
    cudaEventElapsedTime(&regularMs, start, stop);
    
    // Capture graph
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    for (int k = 0; k < KERNELS; k++) {
        tinyKernel<<<1, 256, 0, stream>>>(d_data);
    }
    cudaStreamEndCapture(stream, &graph);
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // Benchmark graph launches
    cudaEventRecord(start);
    for (int iter = 0; iter < ITERATIONS; iter++) {
        cudaGraphLaunch(graphExec, stream);
    }
    cudaStreamSynchronize(stream);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float graphMs;
    cudaEventElapsedTime(&graphMs, start, stop);
    
    printf("Launching %d tiny kernels x %d iterations:\n", KERNELS, ITERATIONS);
    printf("Regular launches: %.2f ms\n", regularMs);
    printf("Graph launches:   %.2f ms\n", graphMs);
    printf("Speedup: %.1fx\n", regularMs / graphMs);
    
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc cuda_graphs_benchmark.cu -o cuda_graphs_benchmark && ./cuda_graphs_benchmark

## CUDA Graph Node Types

| Node Type | Purpose |
|-----------|--------|
| `cudaGraphAddKernelNode` | Kernel execution |
| `cudaGraphAddMemcpyNode` | Memory copy |
| `cudaGraphAddMemsetNode` | Memory set |
| `cudaGraphAddHostNode` | CPU callback |
| `cudaGraphAddChildGraphNode` | Nested graph |
| `cudaGraphAddEventRecordNode` | Record event |
| `cudaGraphAddEventWaitNode` | Wait for event |

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

Complete these exercises to practice CUDA Graphs concepts:

**Exercise 1: Stream Capture**
- Create a graph using stream capture with multiple dependent kernels
- Launch the graph multiple times and measure performance

**Exercise 2: Graph Node Updates**
- Build a graph manually with kernel nodes
- Update kernel parameters without recreating the graph

**Exercise 3: Nested Graphs**
- Create a child graph for a reusable operation
- Embed it in a parent graph

In [ ]:
%%writefile cuda_graphs_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// Simple kernels for graph construction
__global__ void initKernel(float* data, int n, float value) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = value;
}

__global__ void scaleKernel(float* data, int n, float scale) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] *= scale;
}

__global__ void addKernel(float* a, float* b, float* c, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) c[idx] = a[idx] + b[idx];
}

__global__ void reductionKernel(float* input, float* output, int n) {
    __shared__ float sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? input[idx] : 0.0f;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) atomicAdd(output, sdata[0]);
}

// =============================================================================
// Exercise 1: Stream Capture
// TODO: Create a graph using stream capture
// =============================================================================
void exercise1_StreamCapture() {
    printf("=== Exercise 1: Stream Capture ===\n");
    
    int n = 1024 * 1024;
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    float *d_a, *d_b, *d_c;
    CHECK_CUDA(cudaMalloc(&d_a, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_c, n * sizeof(float)));
    
    cudaStream_t stream;
    CHECK_CUDA(cudaStreamCreate(&stream));
    
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    // TODO: Exercise 1a - Begin stream capture
    // Hint: cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
    
    // TODO: Exercise 1b - Launch kernels into the stream (they get captured)
    // Initialize arrays
    initKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, n, 1.0f);
    initKernel<<<numBlocks, blockSize, 0, stream>>>(d_b, n, 2.0f);
    
    // Scale arrays
    scaleKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, n, 2.0f);
    scaleKernel<<<numBlocks, blockSize, 0, stream>>>(d_b, n, 3.0f);
    
    // Add arrays
    addKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, d_b, d_c, n);
    
    // TODO: Exercise 1c - End capture and get graph
    // Hint: cudaStreamEndCapture(stream, &graph);
    CHECK_CUDA(cudaStreamEndCapture(stream, &graph));
    
    // Instantiate graph
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // TODO: Exercise 1d - Launch graph multiple times and time it
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaEventCreate(&start));
    CHECK_CUDA(cudaEventCreate(&stop));
    
    int numIterations = 100;
    
    CHECK_CUDA(cudaEventRecord(start, stream));
    for (int i = 0; i < numIterations; i++) {
        CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
    }
    CHECK_CUDA(cudaEventRecord(stop, stream));
    CHECK_CUDA(cudaStreamSynchronize(stream));
    
    float graphTime;
    CHECK_CUDA(cudaEventElapsedTime(&graphTime, start, stop));
    
    // Compare with non-graph version
    CHECK_CUDA(cudaEventRecord(start, stream));
    for (int i = 0; i < numIterations; i++) {
        initKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, n, 1.0f);
        initKernel<<<numBlocks, blockSize, 0, stream>>>(d_b, n, 2.0f);
        scaleKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, n, 2.0f);
        scaleKernel<<<numBlocks, blockSize, 0, stream>>>(d_b, n, 3.0f);
        addKernel<<<numBlocks, blockSize, 0, stream>>>(d_a, d_b, d_c, n);
    }
    CHECK_CUDA(cudaEventRecord(stop, stream));
    CHECK_CUDA(cudaStreamSynchronize(stream));
    
    float noGraphTime;
    CHECK_CUDA(cudaEventElapsedTime(&noGraphTime, start, stop));
    
    printf("Graph time for %d iterations: %.3f ms\n", numIterations, graphTime);
    printf("No-graph time: %.3f ms\n", noGraphTime);
    printf("Speedup: %.2fx\n\n", noGraphTime / graphTime);
    
    // Cleanup
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaEventDestroy(start));
    CHECK_CUDA(cudaEventDestroy(stop));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFree(d_a));
    CHECK_CUDA(cudaFree(d_b));
    CHECK_CUDA(cudaFree(d_c));
}

// =============================================================================
// Exercise 2: Graph Node Updates
// TODO: Update kernel parameters without recreating the graph
// =============================================================================
void exercise2_GraphUpdate() {
    printf("=== Exercise 2: Graph Node Updates ===\n");
    
    int n = 1024;
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    float *d_data;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    
    // Create graph manually
    cudaGraph_t graph;
    CHECK_CUDA(cudaGraphCreate(&graph, 0));
    
    // Add init kernel node
    cudaGraphNode_t initNode;
    cudaKernelNodeParams initParams = {0};
    
    float initValue = 5.0f;
    void* initArgs[] = {&d_data, &n, &initValue};
    
    initParams.func = (void*)initKernel;
    initParams.gridDim = dim3(numBlocks);
    initParams.blockDim = dim3(blockSize);
    initParams.sharedMemBytes = 0;
    initParams.kernelParams = initArgs;
    initParams.extra = NULL;
    
    CHECK_CUDA(cudaGraphAddKernelNode(&initNode, graph, NULL, 0, &initParams));
    
    // Instantiate
    cudaGraphExec_t graphExec;
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Launch with initial value
    CHECK_CUDA(cudaGraphLaunch(graphExec, 0));
    CHECK_CUDA(cudaDeviceSynchronize());
    
    float h_result;
    CHECK_CUDA(cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost));
    printf("Initial value: %.1f\n", h_result);
    
    // TODO: Exercise 2a - Update the kernel parameter (change init value)
    // Hint: Update initValue and use cudaGraphExecKernelNodeSetParams
    float newValue = 10.0f;
    initArgs[2] = &newValue;
    initParams.kernelParams = initArgs;
    
    CHECK_CUDA(cudaGraphExecKernelNodeSetParams(graphExec, initNode, &initParams));
    
    // Launch with updated value
    CHECK_CUDA(cudaGraphLaunch(graphExec, 0));
    CHECK_CUDA(cudaDeviceSynchronize());
    
    CHECK_CUDA(cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost));
    printf("Updated value: %.1f\n", h_result);
    printf("Status: %s\n\n", (h_result == 10.0f) ? "PASS" : "NEEDS WORK");
    
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaFree(d_data));
}

// =============================================================================
// Exercise 3: Nested Graphs (Child Graph)
// TODO: Create a child graph and embed in parent
// =============================================================================
void exercise3_NestedGraphs() {
    printf("=== Exercise 3: Nested Graphs ===\n");
    
    int n = 1024;
    int blockSize = 256;
    int numBlocks = (n + blockSize - 1) / blockSize;
    
    float *d_data;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    
    // TODO: Exercise 3a - Create a child graph for "double and add 1" operation
    cudaGraph_t childGraph;
    CHECK_CUDA(cudaGraphCreate(&childGraph, 0));
    
    // Scale by 2
    cudaGraphNode_t scaleNode;
    cudaKernelNodeParams scaleParams = {0};
    float scaleVal = 2.0f;
    void* scaleArgs[] = {&d_data, &n, &scaleVal};
    scaleParams.func = (void*)scaleKernel;
    scaleParams.gridDim = dim3(numBlocks);
    scaleParams.blockDim = dim3(blockSize);
    scaleParams.kernelParams = scaleArgs;
    CHECK_CUDA(cudaGraphAddKernelNode(&scaleNode, childGraph, NULL, 0, &scaleParams));
    
    // TODO: Exercise 3b - Create parent graph
    cudaGraph_t parentGraph;
    CHECK_CUDA(cudaGraphCreate(&parentGraph, 0));
    
    // Initialize data in parent
    cudaGraphNode_t initNode;
    cudaKernelNodeParams initParams = {0};
    float initVal = 5.0f;
    void* initArgs[] = {&d_data, &n, &initVal};
    initParams.func = (void*)initKernel;
    initParams.gridDim = dim3(numBlocks);
    initParams.blockDim = dim3(blockSize);
    initParams.kernelParams = initArgs;
    CHECK_CUDA(cudaGraphAddKernelNode(&initNode, parentGraph, NULL, 0, &initParams));
    
    // TODO: Exercise 3c - Add child graph as a node (depends on init)
    cudaGraphNode_t childNode;
    cudaGraphNode_t dependencies[] = {initNode};
    CHECK_CUDA(cudaGraphAddChildGraphNode(&childNode, parentGraph, dependencies, 1, childGraph));
    
    // Instantiate parent graph
    cudaGraphExec_t graphExec;
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, parentGraph, NULL, NULL, 0));
    
    // Launch
    CHECK_CUDA(cudaGraphLaunch(graphExec, 0));
    CHECK_CUDA(cudaDeviceSynchronize());
    
    float h_result;
    CHECK_CUDA(cudaMemcpy(&h_result, d_data, sizeof(float), cudaMemcpyDeviceToHost));
    
    // Expected: init(5) -> scale(2) = 10
    printf("Result: %.1f (expected: 10.0)\n", h_result);
    printf("Status: %s\n\n", (h_result == 10.0f) ? "PASS" : "NEEDS WORK");
    
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(parentGraph));
    CHECK_CUDA(cudaGraphDestroy(childGraph));
    CHECK_CUDA(cudaFree(d_data));
}

int main() {
    printf("\n╔══════════════════════════════════════════════════════════════╗\n");
    printf("║              CUDA Graphs Exercises                           ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    exercise1_StreamCapture();
    exercise2_GraphUpdate();
    exercise3_NestedGraphs();
    
    printf("═══════════════════════════════════════════════════════════════\n");
    printf("Explore the code to understand CUDA Graphs patterns!\n");
    printf("═══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o cuda_graphs_exercises cuda_graphs_exercises.cu && ./cuda_graphs_exercises

### 🔶 Python/Numba Exercises (Optional)

The following exercises explore graph-like patterns in Python:

```python
from numba import cuda
import numpy as np

# Note: Numba doesn't directly support CUDA Graphs, but you can explore
# similar concepts using streams and dependencies

@cuda.jit
def scale_kernel(data, scale):
    idx = cuda.grid(1)
    if idx < data.size:
        data[idx] *= scale

# Exercise: Create a pipeline using multiple streams
def stream_pipeline():
    """
    Create overlapping operations using streams
    (conceptually similar to graph dependencies)
    """
    n = 1024 * 1024
    stream1 = cuda.stream()
    stream2 = cuda.stream()
    
    # TODO: Use streams to overlap transfers and compute
    # This simulates the dependency management of graphs
    pass

# For true CUDA Graphs support, use CUDA C++ or the cuda-python package
```

**Note**: For production CUDA Graphs usage, CUDA C++ provides the full API.

---

## 🎯 Key Takeaways

```
╔══════════════════════════════════════════════════════════════════╗
║                   CUDA GRAPHS MASTERY                            ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  📋 Core Concept: RECIPE CARDS                                   ║
║     Capture workflow once, execute many times with no overhead   ║
║                                                                  ║
║  📋 Essential Patterns:                                          ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  Stream Capture (easiest):                                 │  ║
║  │    cudaStreamBeginCapture(stream, mode)                    │  ║
║  │    // ... your kernel launches ...                         │  ║
║  │    cudaStreamEndCapture(stream, &graph)                    │  ║
║  │                                                            │  ║
║  │  Manual Construction (full control):                       │  ║
║  │    cudaGraphCreate(&graph, 0)                              │  ║
║  │    cudaGraphAddKernelNode(&node, graph, deps, ...)         │  ║
║  │                                                            │  ║
║  │  Graph Update (change parameters):                         │  ║
║  │    cudaGraphExecKernelNodeSetParams(graphExec, node, ...)  │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  ✅ When to Use CUDA Graphs:                                     ║
║     • Many small kernels launched repeatedly                     ║
║     • ML training loops (same operations each iteration)         ║
║     • Physics simulations (same update pattern each frame)       ║
║     • Any workflow where launch overhead > compute time          ║
║                                                                  ║
║  ⚠️  When NOT to Use:                                            ║
║     • Dynamic workflows (different operations each time)         ║
║     • Few large kernels (launch overhead is negligible)          ║
║     • Single-run workflows (capture overhead not amortized)      ║
║                                                                  ║
║  📊 Performance Potential:                                       ║
║     10-100x reduction in launch overhead                         ║
║     Most impactful for launch-bound workloads                    ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## 🎉 Week 9 Complete: Streams & Concurrency Mastery!

```
Week 9 Journey:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Day 1: Stream Basics        → 🏭 Assembly line - keep GPU stations busy
Day 2: Overlap Transfers    → 🔄 Double buffering - never idle!
Day 3: Multi-Stream         → 🚀 Parallel pipelines - concurrent kernels
Day 4: Events               → ⏱️  Stopwatches & checkpoints - sync & time
Day 5: CUDA Graphs          → 📋 Recipe cards - minimal launch overhead
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

You now have COMPLETE control over GPU concurrency!
```

---

## 🔮 What's Next?

**Week 10: Atomics and Synchronization** - Thread-safe GPU programming!

With streams mastered, next we tackle what happens when threads need to **share data**:

```
Week 9 (Streams):           Week 10 (Atomics):
Operations in parallel      Threads accessing SAME memory

[Stream 1]─────────         Thread 1: read x, add 1, write x
[Stream 2]─────────    →    Thread 2: read x, add 1, write x
[Stream 3]─────────                    ↑ RACE CONDITION! 
                            
                            Solution: atomicAdd(&x, 1)
```

Learn to coordinate thousands of threads safely! 🔒